# CPSC 321
Database Management Systems


### Course Topics
 * Relational Model
 * Basic & Intermediate SQL (MySQL)
 * Logical Database Design (ER, normalization)
 * Physical Database Design (optimization)
 * Transactions & Recovery
 * Security 

### Basic Database Terms & Concepts
1. Persistence
    * Persistence of data past execution and termination of a program
2. Database (DB)
    * Some form of data storage and organization combined into a structure
3. Database Management System (DBMS)
    * A piece of software managing any number of databases (e.g. Gonzaga MySQL server managing every student's database)
4. DBMS Supported User Operations
    * Basic, high level support for any DBMS.
        * Design an organizational structure ("schema")
        * Update (add, modify, remove) data from an existing schema
        * Query data from an existing schema (using a "query language")
        * Manage access to DB components
5. Relational DBMS (RDBMS/DBMS)
    * Uses relational model to store data (tables of rows & columns)
    * Typically uses SQL as the query language
6. SQL command (sub) languages
    * SQL = "Structured Query Language"
    * DDL = "Data Definition Language" 
        * used for creation of tables
        * CREATE, ALTER, DROP...
    * DML = "Data Manipulation Language"
        * used for modification of table data
        * INSERT, UPDATE, DELETE...
    * DQL = "Data Query Language"
        * Otherwise known as SQL, used for search and selection of table data
        * SELECT, WITH...
    * DCL = "Data Control Language"
        * Used for permission management
        * GRANT, REVOKE...
7. Transaction Processing Systems (OLTP - Online Transaction Processing)
    * Supports daily operations of businesses
    * DML oriented operations - INSERT, UPDATE, DELETE
    * Write heavy database, with some simple queries as well.
    * Fast responses with lots of concurrent operations
8. Analytics (OLAP - Online Analytical Processing)
    * Supports decision making over a wide sample
    * DQL oriented operations - SELECT, WITH
    * Read heavy database, involves complex queries
9. Transactional Concurrency
    * Lots of transactions happening simultaneously
10. ACID propeties
    * Atomicity - All or nothing, either complete the transaction or end it with no effects on the database
    * Concistency - Transactions dont violate DB constraints
    * Isolation - Two transactions dont impact each other, independent execution of transactions
    * Durability - Transactins are reflected by an updated DB

### Running Example
Database for a fictional e-scooter/e-bike company
* Based loosely on GBFS
* For examples & some homework

Data Studied:
* Vehicle types
* Real-time vehicle data
* Basic customer info
* Price plans
* Customer trip info

### Relations
Vehicle Type
|vt_id (int)|make(string)|model(string)|
|-----|----|-----|
|1|segway|Ninebot|
|2|Lime-S|Gen4|  

R(a<sub>1</sub>, a<sub>z</sub>,...,a<sub>n</sub>)  
VehicleType(vt_id, make, model)

Set - Unordered, no duplicates  
List - Ordered, duplicates allowed  
Bag - Unordered, duplicates allowed  __*USED BY MOST DBMS*__

Relation "schema" =
* Relation Name
* Set of attributes
    * name
    * "domain"
Relation "Instance"=
* A set of "tuples" (rows, records)

__Informally, schema + instance = "Table"__
Many instances of a schema possible, only one in a relation (at a point in time)

"Database Schema" = set of all relation schemas in the database (across all tables)  
"Database Instance" = set of instances of its relations (across all tables)

"Arity" (Degree) = # of attributes, # of columns  
"Cardinality" = # of tuples, # of rows

A domain (attribute) is both: 
* A data (storage) type (int, string, double, etc)
* *OPTIONAL* A constraint on allowable values for the attribute
    * Example: Storing the age of a customer (int), and constraining the range from 0-120
    * "Enumeration" is used to constrain an attribute value to fit an option from a list of allowable values.

We assume "atomic" data types in this class:
* No collection values (no lists, sets, bags)
    * Collection values in instances can slow down queries and create tricky issues to navigate

Our schemas will be in 1st normal form (1NF), which follow the rules above.

NULL is a special value:
* Can go almost anywhere other values can go
* Means either "unknown" or "not applicable"

### Keys
A "key" is an attribute, or set of attributes ("composite key"), with unique values
* If we decide vt_id is a key, then all rows must contain a unique vt_id value.

If we insert:
|vt_id (int)|make(string)|model(string)|
|-|-----|----|
|1|Lime-S|Gen5|

The database will reject the update since it contains a duplicate key.
We usually underline key values when drawing tables.

__Exercise__  
Q: What are the potential keys?

VehicleType(vt_id,make,model)  
Key: vt_id

Vehicle(v_id,vt_id,latitude,longitude)  
Key: v_id

Trip(v_id, customer_id, start_date, start_time, total_cost)  
Key: (v_id, c_id, start_date, start_time), (c_id, start_date, start_time), (v_id, start_date, start_time)  
It depends on the constraints of the database and goals/constraints of the business.

A relation can have multiple keys:
* Customer(c_id, email, ssn, ...)
    * Each attribute is its own key
    * A table can have one or more "candidate" keys
    * Stay minimal on keys, particularly composites.

In a DBMS, each table has a "primary" key (PK).
* Selected from candidate keys
* When we designate an attribute as the primary key:
    * DBMS enforces the uniqueness constraint
    * Tells the DB how we are going to store the table in a file.
* CANNOT BE NULL

__Foreign Keys__

VehicleType
|vt_id|make|model|
|-----|----|-----|
|1|segway|Ninebot|
|2|Lime-S|Gen4| 

Vehicle
|v_id|vt_id|lat|lon|
|-----|----|-----|---|
|10|2|47.66|-114.407|
|15|1|47.66|-117.403|
|20|__4__|47.67|-117.404|

There is a Vehicle instance with vt_id 4, which is not in the VehicleType table.
* "Bad reference"

A foreign key (FK) protects us from bad references.
* FKs state that a column(s) value must come from a primary key in another table.
    * Vehicle.vt_id REFERENCES VehicleType.vt_id (is an FK to)
* Requiring Vehicle rows most refer to VehicleType rows (or else NULL)
* Enforcing FKs helps maintain __"referencial integrity"__

__Check In:__ What are FKs for the following Tables?

VehicleType(vt_id, make, model, default_plan)  
Vehicle(v_id, vt_id, lat, lon)  
PricingPlan(p_id, price_per_min, unlock_price)  
AllowedPlan(vt_id, p_id)  
* Vehicle.vt_id REFERENCES VehicleType.vt_id
* VehicleType.default_plan REFERENCES PricingPlan.p_id
* AllowedPlan.vt_id REFERENCES VehicleType.vt_id
* AllowedPlan.p_id REFERENCES PricingPlan.p_id
* VehicleType.(vt_id, default_plan) REFERENCES AllowedPlan.(vt_id, p_id)

Summary:
1. A FK must reference the entire PK
* If r(__x__,__y__,z), S(__u__,x,y), & S has a FK to R 
* then S.(x,y) references R.(x,y) is a possible FK
* but S.y refs R.y is not
2. A FK Value must contain the corresponding PK value or else be NULL
3. The DBMS enforces FKs

Domains, PKs (& candidate keys), and FKs are examples of __"integrity constraints"__.

__Surrogate Keys__
* Natural Keys
    * Happens most of the time, something plainly obvious as a key
* Artificial Keys
    * Typically used when there is no natural key or there is a large composite key
    * Creating a new key to reference in R from S (z instead of x1, x2, x3, and x4)
* Surrogate Keys
    * Artificial key that DBMS manages

### Schema (Table) Diagram Basics
* Each relation is a separate named block with attributes
* Designate PKs, FKs, and draw connections for FKs